# **CLASS_1**

**What good can ai in robotic fabrication do?**

1. Make intelligent desicions
2. adapt to changes
3. optimise fabrication tasks

**Do we nned ai in robotic fabrication?**

it is usefull for the following applications:
1. computer vision for real-time feedback
2. Reinforcement learning for adaptive control
3. ai driven toolpath optimization
4. generative ai for design & fabrication
5. HUman-Robot collaboration

## Examples of uses of ai in robotic fabrication:

**How do you prototype?**

What-why-how needs to be answered<br>
concepts of operationsdefinition<br>
design<br>
fabrication<br>

**questions to answer:**

where do we get the data?<br>
how do we test? how do we simulate?<br>

it is suggested that we work to get the fabrication data.

# **CLASS_2**

# **CLASS_3**

# **PROJECT**

## 1. BRAINSTORM AND CHOOSING THE TOPIC

1. trasnforming a physiscal object into a dynamic 3d geometry through the use of sensing tools.

2. using ml approaches to make autonomous drones that allows them to detect and avoid obstacles.

3. use ai to control the coordination of robots: <br>
    3.1 use swarms to move objects. <br>
    3.2 use swarm to get from point a to point b using coordinaton between the robot caabilities. <br>
    3.3 use swarms to get robots to coordinate and build something architectural. <br>

4. translating flat drawings into drawings in not planar surfaces.

**INPUTS**

drawing image

**AI PROCESSING**

**ROBOTIC ACTION**